In [5]:
class OldResistor:
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self): # 게터
        return self._ohms
    
    def set_ohms(self, ohms): # 세터
        self._ohms = ohms
        

In [6]:
r0 = OldResistor(50e3)
print('이전: ', r0.get_ohms())
r0.set_ohms(10e3)
print('이후: ', r0.get_ohms())

이전:  50000.0
이후:  10000.0


In [7]:
r0.set_ohms(r0.get_ohms() - 4e3) # 단순연산임에도 코드가 지저분해짐
assert r0.get_ohms() == 6e3 

In [9]:
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
    
r1 = Resistor(50e3)
r1.ohms = 10e3 # 보다 명확해짐

r1.ohms += 5e3




        

In [10]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property
    def voltage(self):
        return self._voltage
    

    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms
    
# voltage property에 대입하면 voltage setter 메서드가 호출되고
# 이 메서드는 current 애트리뷰트를 변경된 전압값에 바꿔 갱신한다


In [11]:
r2 = VoltageResistance(1e3)
print(f'이전: {r2.current: .2f} 암페어')
r2.voltage = 10 # setter 호출 하면 단순 voltage만 바뀌는게 아니라 
print(f'이후: {r2.current:.2f} 암페어 ') # current도 동시에 바뀌게 할수 있다.

이전:  0.00 암페어
이후: 0.01 암페어 


In [13]:
# 프로퍼티에 대해 세터를 지정하면 타입을 검사하거나 클래스 프로퍼티에 전달된 값에 대한 검증을 수행 할 수 있다.

class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'저항 > 0 이어야 합니다. 실제 값: {ohms}')
        self._ohms = ohms

In [15]:
r3 = BoundedResistance(1e3)
r3.ohms = 0 # 잘못된 저항값 에러

ValueError: 저항 > 0 이어야 합니다. 실제 값: 0

In [16]:
BoundedResistance(-5) # 생성자에 잘못된 값을 넘겨도 예외 발생

# why?
# BoundedResistance.__init__() -> Resistor.__init__ 호출
# self.ohms = -5 라는 대입문 실행
# 이때 BoundResistance에 있는 @ohms.setter 메서드 호출
# 객체 생성이 끝나기전에 즉지 저항을 검증하는 코드를 실행

ValueError: 저항 > 0 이어야 합니다. 실제 값: -5

In [17]:
# @property를 활용해 부모 클래스에 정의된 애트리뷰트를 불변으로 만들 수도 있다.

class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'): # 생성할때는 없으니 통과 , 이후는 안됨
            raise AttributeError("OHms는 불변 객체 입니다.")
    
        self._ohms = ohms
    

In [18]:
r4 = FixedResistance(1e3)
r4.ohms = 2e3

AttributeError: OHms는 불변 객체 입니다.

In [19]:
class MysteriousResistor(Resistor):
    @property
    def ohms(self): # getter에서 예기치 못한 동작을 수행하지않도록 해야한다. 즉 게터는 딱 게터만!
        self.voltage = self._ohms * self.current
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms

In [20]:
r7 = MysteriousResistor(10)
r7.current = 0.01
print(f'이전 : {r7.voltage:.2f}')
r7.ohms # 호출만 햇는데? 값이 바뀐다..?
print(f'이후:{r7.voltage:.2f}')



# @property의 가장 큰 단점은, 애트리뷰트를 처리하는 메서드가 하위 클래스 사이에서만 공유될 수 있다는 점이다.

# 재사용 가능한 프로퍼티 로직을 구현할 떄는 디스크립터를 사용해라!

이전 : 0.00
이후:0.10
